In [42]:
from kafka import KafkaConsumer
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import json
import requests

In [87]:
consumer = KafkaConsumer(bootstrap_servers=['localhost:29092'])
consumer.subscribe(['rtdata'])

df = pd.DataFrame(columns=['timestamp','variable','value','delta','Day sin','Day cos','Week sin','Week cos','Year sin','Year cos'])

day = 24*60*60
week = 7*day
year = (365.2425)*day

for message in consumer:
    if df['timestamp'].size > 60*24*2*2:
        df = df.iloc[2:,:]
    new = pd.DataFrame.from_dict(json.loads(message.value.decode("utf-8")))
    new['timestamp'] = pd.to_datetime(new['timestamp'])
    new['delta'] = new.apply(lambda row: int((row['timestamp'] - datetime.now()).total_seconds()/3600), axis=1)
    seconds = new['timestamp'].map(pd.Timestamp.timestamp)
    new['Day sin'] = np.sin(seconds * (2 * np.pi / day))
    new['Day cos'] = np.cos(seconds * (2 * np.pi / day)) 
    new['Week sin'] = np.sin(seconds * (2 * np.pi / week))
    new['Week cos'] = np.cos(seconds * (2 * np.pi / week))
    new['Year sin'] = np.sin(seconds * (2 * np.pi / year))
    new['Year cos'] = np.cos(seconds * (2 * np.pi / year))
    df = pd.concat([df,new])
    df = df[df['delta']<24]
    load = np.array(df[df['variable']=='load'].groupby(['delta'])['value'].mean())
    temperature = np.array(df[df['variable']=='temperature'].groupby(['delta'])['value'].mean())
    fourier = np.array(df.groupby(['delta']).mean(numeric_only=True).iloc[:,1:])
    
    if (load.shape[0] == 24) and (temperature.shape[0] == 24):
        data_tbs = np.stack([np.c_[load, np.zeros(len(load)), temperature, fourier]])
        params={'data': data_tbs.tolist()}
        response = requests.post("http://127.0.0.1:5000/forecast", json=json.dumps(params))
        print(json.loads(response.content))



{'predicted_load': '[20218.01171875]'}


KeyboardInterrupt: 

[20766.169921875]


In [71]:
    req = request.json
    data = np.array(req['data'])
    xdim = data.shape[2]
    
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[0])
    
    # Load pickle file with saved model
    with open('model.pkl',"rb") as saved_model:
        model = pickle.load(saved_model)

    # Predict next hour
    output = model.predict(scaled_data)
    to_inverse = np.repeat(output, xdim, axis=-1)
    predicted_load = scaler.inverse_transform(to_inverse)[:,0]
    

array([[ 2.10392880e+04,  0.00000000e+00,  2.02300000e+01,
        -1.00000000e+00, -6.04500013e-13,  6.23489802e-01,
        -7.81831482e-01,  3.65070779e-01, -9.30979767e-01],
       [ 2.10797130e+04,  0.00000000e+00,  1.88970000e+01,
        -9.65925826e-01,  2.58819045e-01,  5.93820186e-01,
        -8.04597780e-01,  3.64403376e-01, -9.31241204e-01],
       [ 2.09174250e+04,  0.00000000e+00,  1.81320000e+01,
        -8.66025404e-01,  5.00000000e-01,  5.63320058e-01,
        -8.26238774e-01,  3.63735785e-01, -9.31502163e-01],
       [ 1.98127500e+04,  0.00000000e+00,  1.74340000e+01,
        -7.07106781e-01,  7.07106781e-01,  5.32032077e-01,
        -8.46724199e-01,  3.63068007e-01, -9.31762643e-01],
       [ 1.82256380e+04,  0.00000000e+00,  1.67900000e+01,
        -5.00000000e-01,  8.66025404e-01,  5.00000000e-01,
        -8.66025404e-01,  3.62400043e-01, -9.32022644e-01],
       [ 1.69309630e+04,  0.00000000e+00,  1.62630000e+01,
        -2.58819045e-01,  9.65925826e-01,  4.672686

In [79]:
json.dumps(params)

'{"data": [[[21039.288, 0.0, 20.23, -1.0, -6.045000127414911e-13, 0.6234898018579884, -0.781831482468624, 0.36507077924685966, -0.9309797667726677], [21079.713, 0.0, 18.897, -0.9659258262905908, 0.25881904509683884, 0.5938201855745959, -0.8045977797658607, 0.3644033756301696, -0.931241203893673], [20917.425, 0.0, 18.132, -0.8660254037900188, 0.49999999999033495, 0.563320058063651, -0.826238774315975, 0.36373578479142255, -0.9315021625644075], [19812.75, 0.0, 17.434, -0.7071067811878813, 0.7071067811852138, 0.5320320765142446, -0.8467241992289704, 0.3630680070736112, -0.9317626426507967], [18225.638, 0.0, 16.79, -0.5000000000062043, 0.8660254037808567, 0.5000000000008863, -0.8660254037839269, 0.36240004281982446, -0.9320226440190117], [16930.963, 0.0, 16.263, -0.25881904511453874, 0.9659258262858481, 0.46726862827279547, -0.8841153935047507, 0.361731892373247, -0.9322821665354699], [14637.763, 0.0, 18.427, -3.16784416433601e-12, 1.0, 0.4338837391160929, -0.9009688679031248, 0.3610635560